In [1]:
import pydicom
from glob import glob
from copy import deepcopy
from tqdm import tqdm
import numpy as np
import os
import shutil
import SimpleITK as sitk
import time
from natsort import natsorted
from matplotlib import pyplot as plt
from collections import Counter
from pydicom.pixel_data_handlers.util import apply_voi_lut
from PIL import Image
import SimpleITK as sitk
from dataset import CustomImageDataset

In [2]:
import os
gpu_on = True
gpu_number = "0"
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_number
import sys
sys.path.append("../../../../../../CNN_total_Pytorch")
from src.data_set.utils import get_parent_dir_name

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from functools import partial
import random
import pydicom
from albumentations import Compose
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
import SimpleITK as sitk
from glob import glob
import numpy as np
import cv2
from PIL import Image
from tqdm import tqdm
from natsort import natsorted
from matplotlib import pyplot as plt
from scipy.ndimage import zoom
import nibabel as nib
import json
from time import sleep
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

from src.model.train_util.common import mask_gradient
from src.model.train_util.wgan_blend import compute_gradient_penalty_blend, get_blend_images_2d
from src.model.train_util.wgan import compute_gradient_penalty
from src.util.fold_unfold import extract_patch_tensor, combine_region_voting_patches, combine_region_voting_patches_with_patch_weights, PatchSplitModel
from src.util.crop_batch import rotate_and_crop_tensor
from utils import get_z_position, resize_dicom_series, write_series_to_path
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cpu_device = torch.device('cpu')

from src.loss.ssim_psnr import structural_similarity_torch

def get_structural_similarity_numpy(img_1, img_2):
    img_1 = torch.tensor(img_1)[None, None].clamp(-1024, 3071).to(device)
    img_2 = torch.tensor(img_2)[None, None].clamp(-1024, 3071).to(device)
    
    torch_score = structural_similarity_torch(img_1, img_2)
    return torch_score.item()

def get_max_idx(resample_slice, image_array):
    ssim_list = [get_structural_similarity_numpy(resample_slice_array, image_slice) for image_slice in image_array]
    return np.argmax(ssim_list)

## Resample folder 생성

In [3]:
selected_data_folder = "../data/4_external_data_selected"
resampled_data_folder = "../data/5_external_data_resampled"
dcm_series_folder_list = glob(f"{selected_data_folder}/*/*") #모든 dicom 시리즈 폴더 목록을 가져오기

# for dcm_series_folder in tqdm(dcm_series_folder_list):
#     dcm_basename_list = natsorted(os.listdir(dcm_series_folder))
#     instance_number_list = []
#     for dcm_basename in dcm_basename_list:
#         dcm_path = f"{dcm_series_folder}/{dcm_basename}"
#         dicom_metadata = pydicom.dcmread(dcm_path, stop_before_pixels=True) 
#         instance_number = int(dicom_metadata["InstanceNumber"].value)
#         instance_number_list.append(instance_number)
        
#     if instance_number_list == sorted(instance_number_list):
#         pass
#     else:
#         raise

In [5]:
def check_is_dcm_folder(folder_path):
    """
    지정된 폴더의 1단계에서 DICOM 파일이 있는지 확인합니다.
    :param folder_path: 확인할 폴더 경로
    :return: DICOM 파일이 있으면 True, 없으면 False
    """
    is_dcm = False
    # os.walk 사용 (1단계 순회)
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.dcm'):  # DICOM 파일인지 확인 (.dcm 확장자)
                is_dcm = True
                return is_dcm  # DICOM 파일 발견 즉시 반환
        break  # 1단계만 순회하기 위해 종료
    return is_dcm

dcm_series_folder_list = glob(f"{selected_data_folder}/*/*") #모든 dicom시리즈의 폴더목록 가져오기
dcm_series_folder_list = [dcm_series_folder for dcm_series_folder in dcm_series_folder_list
                         if os.path.isdir(dcm_series_folder)]
dcm_series_folder_list = [dcm_series_folder for dcm_series_folder in dcm_series_folder_list
                         if check_is_dcm_folder(dcm_series_folder)]
print(len(dcm_series_folder_list))
target_hw = (512, 512)
for index, dicom_series_path in enumerate(dcm_series_folder_list):
    
    folder_basename = get_parent_dir_name(dicom_series_path, level=1)
    slice_thickness = float(get_parent_dir_name(dicom_series_path, level=0))
    resized_slice_thickness = slice_thickness * 3
    
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(dicom_series_path)
    original_sample_path = dicom_names[0]
    reader.SetFileNames(dicom_names)
    reader.MetaDataDictionaryArrayUpdateOn()
    reader.LoadPrivateTagsOn()
    
    image = reader.Execute()
    image_size = image.GetSize()
    resize_factor_hw = target_hw / np.array(image_size)[:2]

    resize_factor_list = (*resize_factor_hw, (1 / 3))

    image_sample = image[:, :, 0]
    dicom_file = pydicom.dcmread(dicom_names[0], force=True)
    dicom_file_next = pydicom.dcmread(dicom_names[2], force=True)
    image_position_z_diff = np.abs(get_z_position(dicom_file_next) - get_z_position(dicom_file))
    original_z_spacing = image_position_z_diff
    slice_num = len(dicom_names)
    target_path = f"{resampled_data_folder}/{folder_basename}/{resized_slice_thickness:.3f}"
    
    resized_image = resize_dicom_series(image, resize_factor_list, is_label=False, resize_method="sitk")
    resized_image_size = resized_image.GetSize()
    print(f"sample path: {index} {original_sample_path}")
    print("Image size:", *image_size, "=> resample size:", *resized_image_size)
    print("resize image:", resized_image_size[0], resized_image_size[1], resized_image_size[2])
    print(f"slice_thickness: {slice_thickness} => {resized_slice_thickness}")
    print(f"image_direction: {resized_image.GetDirection()}")
    print(f"target_path: {target_path}")
    
    write_series_to_path(reader=reader,
                     target_image=resized_image, 
                     original_sample_path=original_sample_path, 
                     target_path=target_path, 
                     slice_thickness=resized_slice_thickness,
                     inverse_instance_num=False)

23
sample path: 0 ../data/4_external_data_selected/1.3.12.2.1107.5.1.4.73271.30000024070120571734000018233/2.000/0001.dcm
Image size: 512 512 105 => resample size: 512 512 35
resize image: 512 512 35
slice_thickness: 2.0 => 6.0
image_direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
target_path: ../data/5_external_data_resampled/1.3.12.2.1107.5.1.4.73271.30000024070120571734000018233/6.000
sample path: 1 ../data/4_external_data_selected/1.3.12.2.1107.5.1.4.73271.30000024071904171261200011263/3.000/0001.dcm
Image size: 512 512 57 => resample size: 512 512 19
resize image: 512 512 19
slice_thickness: 3.0 => 9.0
image_direction: (0.999183224011328, 0.02396745617098902, 0.03253376552169164, -0.02457034139062425, 0.999531335135537, 0.01825947442068212, -0.03208088493599995, -0.019043926246000018, 0.9993038305214588)
target_path: ../data/5_external_data_resampled/1.3.12.2.1107.5.1.4.73271.30000024071904171261200011263/9.000
sample path: 2 ../data/4_external_data_selected/1.3.12.2.1107

sample path: 20 ../data/4_external_data_selected/1.3.12.2.1107.5.99.2.46822.30000024093000304744900281082/2.000/0001.dcm
Image size: 512 512 87 => resample size: 512 512 29
resize image: 512 512 29
slice_thickness: 2.0 => 6.0
image_direction: (1.0, -1.2246468e-16, 0.0, 1.2246468e-16, 1.0, 0.0, 0.0, 0.0, 1.0)
target_path: ../data/5_external_data_resampled/1.3.12.2.1107.5.99.2.46822.30000024093000304744900281082/6.000
sample path: 21 ../data/4_external_data_selected/1.3.12.2.1107.5.99.2.46822.30000024093000304744900285679/2.000/0001.dcm
Image size: 512 512 90 => resample size: 512 512 30
resize image: 512 512 30
slice_thickness: 2.0 => 6.0
image_direction: (1.0, -1.2246468e-16, 0.0, 1.2246468e-16, 1.0, 0.0, 0.0, 0.0, 1.0)
target_path: ../data/5_external_data_resampled/1.3.12.2.1107.5.99.2.46822.30000024093000304744900285679/6.000
sample path: 22 ../data/4_external_data_selected/1.3.12.2.1107.5.99.2.46822.30000024093000304744900286776/2.000/0001.dcm
Image size: 512 512 87 => resample size